# Imports

https://hallab.cs.dal.ca/images/0/00/Minh2015.pdf

In [1]:
import gym
from gym import wrappers
import numpy as np
import cv2
import tensorflow as tf
import os
import collections
import datetime
from statistics import mean
from ipywidgets import widgets
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from pathlib import Path
# Implementation
import model

# Helper functions

In [3]:
import numpy as np
import random
from collections import deque

import matplotlib.pyplot as plt
from PIL import Image
#import imageio

import os
import gym
#from tqdm import tqdm

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

In [3]:
def make_video(env, TrainNet):
    env = wrappers.Monitor(env, os.path.join(os.getcwd(), "videos"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    while not done:
        action = TrainNet.get_action(observation, 0)
        observation, reward, done, _ = env.step(action)
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))

In [6]:

class Agent(object):
    def __init__(self, enviroment, optimizer, image_shape):
        
        # Initialize atributes
        self._action_size = enviroment.action_space.n
        self._optimizer = optimizer
        self._image_shape = image_shape
        self.enviroment = enviroment
        
        self.expirience_replay = deque(maxlen=100000)
        
        # Initialize discount and exploration rate
        self.gamma = 0.6
        self.epsilon = 0.1
        
        # Build networks
        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        self.alighn_target_model()

    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.append((state, action, reward, next_state, terminated))
    
    def _update_epsilon(self):
        self.epsilon -= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
    
    def _build_compile_model(self):
        model = Sequential()
        model.add(Conv2D(32, 8, strides=(4, 4), padding="valid",activation="relu", 
                              input_shape = self._image_shape))
        model.add(Conv2D(64, 4, strides=(2, 2), padding="valid", activation="relu",
                              input_shape = self._image_shape))
        model.add(Conv2D(64, 3, strides=(1, 1), padding="valid",activation="relu",
                              input_shape = self._image_shape))
        model.add(Flatten())
        model.add(Dense(512, activation="relu"))
        model.add(Dense(self._action_size))
        huber = Huber()
        model.compile(loss = huber,
                           optimizer=self._optimizer,
                           metrics=["accuracy"])
        return model

    def alighn_target_model(self):
        self.target_network.set_weights(self.q_network.get_weights())
    
    def act(self, frame):
        if np.random.rand() <= self.epsilon:
            return self.enviroment.action_space.sample()
        
        frame = np.expand_dims(np.asarray(frame).astype(np.float64), axis=0)
        
        q_values = self.q_network.predict(frame)
        return np.argmax(q_values[0])
    

    def retrain(self, batch_size):
        minibatch = random.sample(self.expirience_replay, batch_size)
        
        for state, action, reward, next_state, terminated in minibatch:
            
            state = np.expand_dims(np.asarray(state).astype(np.float64), axis=0)
            next_state = np.expand_dims(np.asarray(next_state).astype(np.float64), axis=0)
            
            target = self.q_network.predict(state)
            
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            
            self.q_network.fit(state, target, epochs=1, verbose=0)

# Hyperparameter definition

## File system

In [4]:
GAME_NAME = "BreakoutDeterministic-v4"
NUMBER_OF_FRAMES = 5

enviroment = gym.make(GAME_NAME).env

In [5]:
enviroment.reset()

frames = []
for _ in range(NUMBER_OF_FRAMES):
    enviroment.step(enviroment.action_space.sample())
    frames.append(enviroment.ale.getScreenRGB())
img_processor.plot_frames(frames)

NameError: name 'img_processor' is not defined

In [7]:
optimizer = Adam(learning_rate=0.01)

state = enviroment.reset()

agent = Agent(enviroment, optimizer, state.shape)

batch_size = 32
num_of_episodes = 1000
timesteps_per_episode = 1000
agent.q_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 51, 39, 32)        6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 18, 64)        32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 16, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 22528)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               11534848  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 11,612,836
Trainable params: 11,612,836
Non-trainable params: 0
____________________________________________

In [8]:
for e in range(0, num_of_episodes):
    # Reset the enviroment
    state = enviroment.reset()

    # Initialize variables
    reward = 0
    terminated = False

    for timestep in range(timesteps_per_episode):
        state = img_processor.process_env_state(state)

        # Run Action
        action = agent.act(state)

        # Take action    
        next_state, reward, terminated, info = enviroment.step(action) 
        next_state = img_processor.process_env_state(next_state)

        agent.store(state, action, reward, next_state, terminated)

        state = next_state

        if terminated:
            agent.alighn_target_model()
            break

        if len(agent.expirience_replay) > batch_size:
            agent.retrain(batch_size)

NameError: name 'img_processor' is not defined

In [71]:
# DQN hyperparameters
gamma = 0.99
copy_step = 45
hidden_units = [250, 250]
max_experiences = 10000
min_experiences = 100
batch_size = 64
lr = 1e-3 #1e-2
epochs = 500 #iterations
total_rewards = np.empty(epochs)
epsilon = 0.99
decay = 0.9999
replay_size=100
eps_min = 0.1

In [53]:
exp_name_train = "train_test_1"
exp_name_target = "target_test_1"

In [54]:
input_shape = (84, 84, 4)

In [55]:
# Model definition
model_train = model.CNN_Model(input_shape, num_states, hidden_units, num_actions)
model_target = model.CNN_Model(input_shape, num_states, hidden_units, num_actions)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 10, 10, 64)        32832     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
dense_16 (Dense)             (None, 8, 8, 512)         33280     
_________________________________________________________________
re_lu_8 (ReLU)               (None, 8, 8, 512)         0         
_________________________________________________________________
dense_17 (Dense)             (None, 8, 8, 6)           3078      
Total params: 114,342
Trainable params: 114,342
Non-trainable params: 0
________________________________________________

In [56]:
# Model instantiation
train_net = model.DQN(exp_name_train, output_dir, model_train, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
target_net = model.DQN(exp_name_target, output_dir, model_target, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)

# New training

In [70]:
buffer = ExperienceReplay(replay_size)
agent = Agent(env, buffer)


optimizer = tf.keras.optimizers.Adam(lr=lr)
total_rewards = []
frame_idx = 0  
best_mean_reward = None

(4, 84, 84)
(4, 84, 84)


In [ ]:
while True:
        frame_idx += 1
        epsilon = max(epsilon*eps_decay, eps_min)

        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)

            mean_reward = np.mean(total_rewards[-100:])

            print("%d:  %d games, mean reward %.3f, (epsilon %.2f)" % (
                frame_idx, len(total_rewards), mean_reward, epsilon))
            
            print("epsilon", epsilon, frame_idx)
            print("reward_100", mean_reward, frame_idx)
            print("reward", reward, frame_idx)

            if best_mean_reward is None or best_mean_reward < mean_reward:
                model.save_model(frame_idx)
                best_mean_reward = mean_reward
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f" % (best_mean_reward))

            if mean_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        if len(buffer) < replay_start_size:
            continue

        batch = buffer.sample(batch_size)
        states, actions, rewards, dones, next_states = batch

        states_v = torch.tensor(states).to(device)
        next_states_v = torch.tensor(next_states).to(device)
        actions_v = torch.tensor(actions).to(device)
        rewards_v = torch.tensor(rewards).to(device)
        done_mask = torch.ByteTensor(dones).to(device)

        state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

        next_state_values = target_net(next_states_v).max(1)[0]

        next_state_values[done_mask] = 0.0

        next_state_values = next_state_values.detach()

        expected_state_action_values = next_state_values * gamma + rewards_v

        loss_t = nn.MSELoss()(state_action_values, expected_state_action_values)

        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()

        if frame_idx % sync_target_frames == 0:
            model_target.load_state_dict(net.state_dict())

# Training

In [57]:
for n in range(epochs):
    epsilon = max(min_epsilon, epsilon * decay)
    total_reward, losses = play_game(env, train_net, target_net, epsilon, copy_step)
    total_rewards[n] = total_reward
    avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
    with summary_writer.as_default():
        tf.summary.scalar('episode reward', total_reward, step=n)
        tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
        tf.summary.scalar('average loss)', losses, step=n)
    if n % 100 == 0:
        print("episode:", n, "episode reward:", total_reward, "eps:", epsilon, "avg reward (last 100):", avg_rewards,
              "episode loss: ", losses)
print("avg reward for last 100 episodes:", avg_rewards)

(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 84, 84)
(4, 

ValueError: Input 0 of layer sequential_8 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: (1, 1)

# Make video

In [ ]:
env = gym.make('CartPole-v0')
make_video(env, target_net)
env.close()

# Tests